In [1]:
using LowLevelFEM
gmsh.initialize()

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodE

In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000371966s, CPU 0.000361s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0082152s, CPU 0.008202s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = Material("body")
prob = Problem([mat], type=:PlaneStress)

Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
K = stiffnessMatrix(prob)
M = massMatrix(prob)

sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  187, 188, 189, 190, 191, 192, 193, 194, 195, 196], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  187, 188, 189, 190, 191, 192, 193, 194, 195, 196], [2.993020228862359e-11, 2.993020228862359e-11, 2.993020228862358e-11, 2.993020228862358e-11, 2.993020228862359e-11, 2.993020228862359e-11, 2.99302022886236e-11, 2.99302022886236e-11, 4.420508840131675e-11, 4.420508840131675e-11  …  6.513183780583524e-11, 6.513183780583524e-11, 6.57312862682614e-11, 6.57312862682614e-11, 6.810895857026154e-11, 6.810895857026154e-11, 5.4986235500860704e-11, 5.4986235500860704e-11, 7.466906578570302e-11, 7.466906578570302e-11], 196, 196)

In [5]:
supp = BoundaryCondition("left", ux=0, uy=0)

BoundaryCondition("left", 0, 0, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [6]:
Tmin = smallestPeriodTime(K, M, support=[supp])
Δt = Tmin / π

1.50747861528746e-8

In [7]:
function fu(x, y, z, t)
    if t < 2Δt
        return 0.0
    elseif t < 4Δt
        return 1.0e-6
    else
        return 0.0
    end
end
function ff(x, y, z, t)
    if t < 50Δt
        return 0.0
    elseif t < 52Δt
        return 1.0
    else
        return 0.0
    end
end
su = ScalarField(prob, "right", fu, steps=100, tmax=100Δt)
sf = ScalarField(prob, "right", ff, steps=100, tmax=100Δt)

supp2 = BoundaryCondition("right", ux=su)
load = BoundaryCondition("right", fy=sf)

f = loadVector(prob, [load])
u0 = vectorField(prob, "body", [0, 0, 0])
v0 = copy(u0)


VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
showDoFResults(u0, name="u0")
showDoFResults(f, name="f")

2

In [9]:
u, v = CDM(K, M, f, u0, v0, 100, Tmin / π, support=[supp, supp2])

(VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 4.518617954450232e-7 5.287993321567991e-7; 0.0 0.0 … 9.804503100713944e-7 5.028978298722793e-7], [0.0, 1.50747861528746e-8, 3.01495723057492e-8, 4.52243584586238e-8, 6.02991446114984e-8, 7.5373930764373e-8, 9.044871691724759e-8, 1.0552350307012219e-7, 1.205982892229968e-7, 1.356730753758714e-7  …  1.356730753758712e-6, 1.3718055399115864e-6, 1.386880326064461e-6, 1.4019551122173354e-6, 1.41702989837021e-6, 1.4321046845230844e-6, 1.447179470675959e-6, 1.4622542568288334e-6, 1.477329042981708e-6, 1.4924038291345824e-6], Int64[], 100, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 7.8342

In [10]:
showDoFResults(u, :uvec, visible=true)

3

In [11]:
showDoFResults(v, :vvec)

4

In [12]:
e1 = solveStrain(u)

TensorField
[[0.0 0.0 … -3.5979699461157065e-6 -1.2145520273974788e-6; 0.0 0.0 … -9.659529787858404e-7 -3.975096403203136e-6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 2.017740605319382e-6 5.893283907034547e-7], [0.0 0.0 … -1.413375399679041e-6 -2.3347679442812453e-6; 0.0 0.0 … -2.8149201512204605e-6 -2.5112494697107178e-6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 8.396072179553371e-7 1.8667560769953969e-6], [0.0 0.0 … -3.4543986116876595e-6 -6.034887487037695e-6; 0.0 0.0 … -1.7676329615033678e-6 -1.129170149178594e-6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 1.5815831032576924e-6 2.4042382916588213e-6], [0.0 0.0 … -8.849433078257727e-7 7.94312959864981e-7; 0.0 0.0 … 1.3629229623846177e-8 5.300760544813435e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -2.9075166713077166e-7 1.182968290828107e-7], [0.0 0.0 … -3.018292603656656e-6 -4.859781004221279e-6; 0.0 0.0 … -5.055923050079602e-7 -6.655393025632157e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 1.0708542793690143e-7 2.648064521324216e-6], [0.0 0.0 … 1.04021386995305

In [13]:
showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


5

In [14]:
e2 = solveStrain(u, DoFResults=true)

TensorField
[0.0 0.0 … -2.8318980450185894e-6 -1.626650418160707e-6; 0.0 0.0 … -1.8588852603533965e-6 -1.873083025027767e-6; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -7.650713063923125e-8 -3.355957006423845e-7]

In [15]:
showDoFResults(e2, :e)

6

In [16]:
e3 = elementsToNodes(e1)

TensorField
[0.0 0.0 … -2.8318980450185894e-6 -1.626650418160707e-6; 0.0 0.0 … -9.294426301766983e-7 -9.365415125138835e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -7.650713063923125e-8 -3.355957006423845e-7]

In [17]:
showDoFResults(e3, :e)

7

In [18]:
s1 = solveStress(u)

TensorField
[[0.0 0.0 … -0.863955469451552 -0.2775200643232205; 0.0 0.0 … -0.14860815058243698 -0.6115532928004825; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.3466127104052889 -0.6463883109656372; 0.0 0.0 … -0.4330646386493016 -0.38634607226318735; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.7747664176839003 -1.2983270428763871; 0.0 0.0 … -0.2719435325389797 -0.17371848448901447; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.09141409856846171 0.10400248165879543; 0.0 0.0 … 0.0020968045575147963 0.081550162227899; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.48082738947593184 -1.1550531577762297; 0.0 0.0 … -0.07778343153968618 -0.10239066193280241; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.21172525733592681 0.1005864998194084; 0.0 0.0 … -0.16616073118390237 0.04958637602008994; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -0.13488377258104678 -0.254325652050939; 0.0 0.0 … -0.5488377325042049 -0.5918343084012341; … 

In [19]:
showStressResults(s1, :s)

8

In [20]:
s2 = solveStress(u, DoFResults=true)

TensorField
[0.0 0.0 … -0.5961230687957344 -0.36978185251482243; 0.0 0.0 … -0.1429911738733382 -0.14408330961752053; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [21]:
showDoFResults(s2, :s)

9

In [22]:
err1 = fieldError(e1)

TensorField
[0.0 0.0 … 5.34603373088869e-7 6.588493976519959e-7; 0.0 0.0 … 6.807154988110406e-8 2.3632968085643915e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.715200965589532e-7 1.295446911912315e-6]

In [23]:
showDoFResults(err1, :tensor)

10

In [24]:
u2, v2 = HHT(K, M, f, [supp, supp2], u0, v0, 100, Tmin / π)

(VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 2.739801394983496e-7 3.170799366195893e-7; 0.0 0.0 … 9.094107764145928e-7 9.50700779724724e-7], [0.0, 1.50747861528746e-8, 3.01495723057492e-8, 4.52243584586238e-8, 6.02991446114984e-8, 7.5373930764373e-8, 9.044871691724759e-8, 1.0552350307012219e-7, 1.205982892229968e-7, 1.356730753758714e-7  …  1.356730753758712e-6, 1.3718055399115864e-6, 1.386880326064461e-6, 1.4019551122173354e-6, 1.41702989837021e-6, 1.4321046845230844e-6, 1.447179470675959e-6, 1.4622542568288334e-6, 1.477329042981708e-6, 1.4924038291345824e-6], Int64[], 100, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.93184

In [25]:
showDoFResults(u2, :uvec)

11

In [26]:
showDoFResults(v2, :vvec)

12

In [27]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [28]:
gmsh.finalize()